# IF3270 Pembelajaran Mesin | Tugas Besar - Feedforward Neural Network

Group Members:
- Maximilian Sulistiyo (13522061)
- Marvel Pangondian (13522075)
- Abdullah Mubarak (13522101)

In this project, we implement a custom built Feedforward Neural Network with no high-level libraries. The goal in this project is to be able to create a custom FFNN that is able to specify the type of activation function on each layer, the type of loss function, and how many neurons in each layer. We will also compare our algorithm with other built in algorithm (the sklearn MLP)

## Import Libraries

In [1]:
import numpy as np
import pandas as pd
from ann import NeuralNetwork, one_hot, get_accuracy
from dense_layer import DenseLayer
from activations import *
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report
from activations import tanh, sigmoid, relu

## Import Dataset

### Fetch dataset

In [2]:
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

c:\Python310\lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
X.shape

(70000, 784)

In [4]:
y.shape

(70000,)

### Copy dataset and normalize

In [5]:
X_original = X.copy()
X_original = X_original/255.0
y_original = y.copy()
y_original = y_original.astype(int)

### Split dataset

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_original, y_original, train_size=60000, test_size=10000, stratify=y)

In [11]:
y_train_oh = one_hot(y_train)
y_test_oh = one_hot(y_test)

## Example of Training Model

In [8]:
model = NeuralNetwork('mse')
model.add_layer(DenseLayer(output_size=128, activation=relu, init="Xavier"))
model.add_layer(DenseLayer(output_size=64, activation=relu, init="Xavier"))
model.add_layer(DenseLayer(output_size=10, activation=sigmoid, init="Xavier"))

model.train(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    learning_rate=0.05,
    verbose=1,
)

Epoch 1/100 - 1.87s - loss: 0.0989
Epoch 2/100 - 1.79s - loss: 0.0797
Epoch 3/100 - 1.73s - loss: 0.0615
Epoch 4/100 - 1.74s - loss: 0.0482
Epoch 5/100 - 1.76s - loss: 0.0393
Epoch 6/100 - 1.81s - loss: 0.0328
Epoch 7/100 - 1.82s - loss: 0.0284
Epoch 8/100 - 1.87s - loss: 0.0255
Epoch 9/100 - 1.86s - loss: 0.0233
Epoch 10/100 - 1.82s - loss: 0.0217
Epoch 11/100 - 1.91s - loss: 0.0205
Epoch 12/100 - 2.10s - loss: 0.0195
Epoch 13/100 - 1.82s - loss: 0.0187
Epoch 14/100 - 1.97s - loss: 0.0180
Epoch 15/100 - 2.12s - loss: 0.0174
Epoch 16/100 - 2.15s - loss: 0.0168
Epoch 17/100 - 1.93s - loss: 0.0163
Epoch 18/100 - 1.88s - loss: 0.0159
Epoch 19/100 - 1.94s - loss: 0.0155
Epoch 20/100 - 1.90s - loss: 0.0152
Epoch 21/100 - 1.94s - loss: 0.0148
Epoch 22/100 - 2.04s - loss: 0.0145
Epoch 23/100 - 1.97s - loss: 0.0142
Epoch 24/100 - 1.86s - loss: 0.0140
Epoch 25/100 - 1.74s - loss: 0.0137
Epoch 26/100 - 1.66s - loss: 0.0135
Epoch 27/100 - 1.79s - loss: 0.0132
Epoch 28/100 - 1.76s - loss: 0.0130
E

{'loss': [0.09887546385797048,
  0.07965197314605793,
  0.0615380863021446,
  0.04818587192802164,
  0.03929758146076872,
  0.03282052751124533,
  0.028445561648181978,
  0.025452193065893945,
  0.023327418723962417,
  0.021735940598507753,
  0.02049672342264637,
  0.019509111231029715,
  0.018678546067509096,
  0.017971643722024516,
  0.01736136759714431,
  0.016826387746934738,
  0.016344665049137404,
  0.015913710620197034,
  0.015526006377868887,
  0.015166287754794089,
  0.014829212624838406,
  0.014517471090078487,
  0.014232228385742892,
  0.013962852874065573,
  0.01370337689221501,
  0.013468673084950013,
  0.013237415466681966,
  0.013020250379980659,
  0.01282030265402161,
  0.012620597833985463,
  0.012428999676826877,
  0.012247858386089439,
  0.012077063356956897,
  0.011912167725123728,
  0.011748543002166293,
  0.011605521211064694,
  0.01144576889549084,
  0.011295389455574333,
  0.011158907111500624,
  0.011017806494414376,
  0.01089351182873042,
  0.0107556341812861,

In [9]:
predictions = model.predict(X_test)
pred_classes = np.argmax(predictions, axis=1)
accuracy = accuracy_score(pred_classes, y_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.9584


## Analysis

### Effect of depth (Number of layers) and Width (Number of neurons per layer)

#### Fixed Depth
- Number of hidden layers : 2
- Test 1 : 64 neurons per layer
- Test 2 : 128 neurons per layer
- Test 3 : 256 neurons per layer

##### Test 1

In [14]:
model_test_1_fixed_depth = NeuralNetwork('mse')
model_test_1_fixed_depth.add_layer(DenseLayer(output_size=128, activation=ReLu(), init="Xavier"))
model_test_1_fixed_depth.add_layer(DenseLayer(output_size=64, activation=ReLu(), init="Xavier"))
model_test_1_fixed_depth.add_layer(DenseLayer(output_size=10, activation=Sigmoid(), init="Xavier"))

model_test_1_fixed_depth.train(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    learning_rate=0.05,
    optimizer="gradient_descent",
    isOne_hot=True,
    verbose=1,
    validation_data=(X_test, y_test)
)

[=================================================>] 100%
Epoch 1/100 - 1.70s - loss: 0.2490 - val_loss: 0.2324
[=================================================>] 100%
Epoch 2/100 - 1.71s - loss: 0.2177 - val_loss: 0.2028
[=================================================>] 100%
Epoch 3/100 - 1.62s - loss: 0.1879 - val_loss: 0.1729
[=================================================>] 100%
Epoch 4/100 - 1.53s - loss: 0.1591 - val_loss: 0.1461
[=================================================>] 100%
Epoch 5/100 - 1.98s - loss: 0.1355 - val_loss: 0.1260
[=================================================>] 100%
Epoch 6/100 - 2.10s - loss: 0.1189 - val_loss: 0.1128
[=================================================>] 100%
Epoch 7/100 - 1.69s - loss: 0.1085 - val_loss: 0.1048
[=================================================>] 100%
Epoch 8/100 - 1.91s - loss: 0.1022 - val_loss: 0.1000
[=================================================>] 100%
Epoch 9/100 - 1.70s - loss: 0.0985 - val_loss:

{'loss': [0.24896870999864829,
  0.21772392982372143,
  0.1878529883690506,
  0.15913406169171143,
  0.13547849554725733,
  0.11888395854282767,
  0.10846971771868423,
  0.10221966329036776,
  0.09845411889933237,
  0.09612035797413274,
  0.09461976627607036,
  0.09361719290638693,
  0.09292223669802564,
  0.09242344049592666,
  0.09205330529661417,
  0.0917698732809284,
  0.09154618515040751,
  0.09136444076693685,
  0.09121272032420445,
  0.09108276012158575,
  0.09096874825612844,
  0.090866563289696,
  0.0907732190541417,
  0.09068650264306288,
  0.09060473894455172,
  0.09052672685408471,
  0.09045152268771581,
  0.09037839559564889,
  0.09030676406045215,
  0.09023614192226592,
  0.09016616182642946,
  0.09009653175515227,
  0.09002701615739579,
  0.08995740807194884,
  0.08988751027646762,
  0.08981716356881375,
  0.0897462286851297,
  0.0896745669906275,
  0.08960207383714722,
  0.08952864436498795,
  0.08945421173968954,
  0.08937866051272636,
  0.08930186656640292,
  0.089223

In [15]:
predictions_test_1_fixed_depth = model_test_1_fixed_depth.predict(X_test)
pred_classes_test_1_fixed_depth = np.argmax(predictions_test_1_fixed_depth, axis=1)
accuracy_test_1_fixed_depth = accuracy_score(pred_classes_test_1_fixed_depth, y_test)
print("Test Accuracy:", accuracy_test_1_fixed_depth)

Test Accuracy: 0.5054


## Compare model with sklearn MLP

In [16]:
mlp = MLPClassifier(hidden_layer_sizes=(128, 64),activation='relu', 
                    solver='adam', max_iter=20, random_state=1, verbose=True)

mlp.fit(X_train, y_train)

Iteration 1, loss = 0.37724840
Iteration 2, loss = 0.15004672
Iteration 3, loss = 0.10562821
Iteration 4, loss = 0.08064147
Iteration 5, loss = 0.06428497
Iteration 6, loss = 0.05121985
Iteration 7, loss = 0.04219482
Iteration 8, loss = 0.03723277
Iteration 9, loss = 0.02986735
Iteration 10, loss = 0.02325205
Iteration 11, loss = 0.02124234
Iteration 12, loss = 0.01549812
Iteration 13, loss = 0.01433403
Iteration 14, loss = 0.01377085
Iteration 15, loss = 0.00927998
Iteration 16, loss = 0.00973711
Iteration 17, loss = 0.01097088
Iteration 18, loss = 0.00895768
Iteration 19, loss = 0.00620646
Iteration 20, loss = 0.00376964


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=20, random_state=1,
              verbose=True)

In [17]:
y_pred_mlp = mlp.predict(X_test)
acc_mlp = accuracy_score(y_test, y_pred_mlp)
print(f"MLPClassifier Test Accuracy: {acc_mlp:.4f}")

MLPClassifier Test Accuracy: 0.9790


In [18]:
print(classification_report(y_test, y_pred_mlp))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       986
           1       0.99      0.99      0.99      1125
           2       0.97      0.98      0.98       999
           3       0.99      0.95      0.97      1020
           4       0.98      0.98      0.98       975
           5       0.96      0.98      0.97       902
           6       0.99      0.99      0.99       982
           7       0.98      0.98      0.98      1042
           8       0.98      0.97      0.97       975
           9       0.95      0.99      0.97       994

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

